<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/QNN_processed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ⬇️ 安裝必要套件（Colab 一次執行）
!pip install --upgrade -q gspread gspread_dataframe

import pandas as pd
import numpy as np
import gspread
from gspread_dataframe import get_as_dataframe
from google.colab import auth
from google.auth import default

# === 🔐 授權 Google API（Colab 內建支援）===
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# === 🔧 參數設定 ===
INTERNAL_URL = 'https://docs.google.com/spreadsheets/d/1p-7J-6lnpsi2i7a9z6ORktCZKXQGOw616GLVP7ynn7k/edit?usp=sharing'
EXTERNAL_URL = 'https://docs.google.com/spreadsheets/d/1Hoguf7PGhJoy0bGzIxf3P7yMHWHuKvdwyd4grxvRHkA/edit?usp=sharing'
EXTERNAL_SHEET_NAME = '測試資料表'

TOP_FEATURES = ['預估年齡', 'ADL_總分_max', 'GDS_總分_max', '六個月內住院次數', '性別_is_male']
LABEL_COLUMN = '死亡標記'

# === 📥 資料載入函數 ===
def load_sheet_df(url, worksheet_name=None):
    sh = gc.open_by_url(url)
    ws = sh.worksheet(worksheet_name) if worksheet_name else sh.get_worksheet(0)
    df = get_as_dataframe(ws, evaluate_formulas=True, na_values=['', 'NA'])
    df = df.dropna(how='all')  # 去除全為空的列
    df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
    return df

# === 📊 執行主流程 ===
print("📥 載入訓練資料（內部）...")
df_internal = load_sheet_df(INTERNAL_URL)

print("📥 載入測試資料（外部）...")
df_external = load_sheet_df(EXTERNAL_URL, worksheet_name=EXTERNAL_SHEET_NAME)

X_internal = df_internal[TOP_FEATURES].copy()
y_internal = df_internal[LABEL_COLUMN]
X_external = df_external[TOP_FEATURES].copy()
y_external = df_external[LABEL_COLUMN]

# === 個別補值規則 ===
print("🧩 根據特徵進行個別補值...")
filling_rules = {
    '預估年齡': X_internal['預估年齡'].mean(),
    'ADL_總分_max': X_internal['ADL_總分_max'].median(),
    'GDS_總分_max': 0,
    '六個月內住院次數': 0,
    '性別_is_male': 0
}
for col, val in filling_rules.items():
    X_internal[col] = X_internal[col].fillna(val)
    X_external[col] = X_external[col].fillna(val)

# === 📤 儲存補值後的資料為 CSV 檔 ===
df_internal_out = X_internal.copy()
df_internal_out['label'] = y_internal.values

df_external_out = X_external.copy()
df_external_out['label'] = y_external.values

df_internal_out.to_csv('processed_internal.csv', index=False)
df_external_out.to_csv('processed_external.csv', index=False)

print("✅ 補值完成，未經標準化，已匯出 CSV！")

# === 🔽 提供下載連結 ===
from google.colab import files
files.download('processed_internal.csv')
files.download('processed_external.csv')

📥 載入訓練資料（內部）...
📥 載入測試資料（外部）...
🧩 根據特徵進行個別補值...
✅ 補值完成，未經標準化，已匯出 CSV！


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>